In [207]:
#Import Dependencies
import pandas as pd

In [208]:
# The path to our CSV file
school_file = "Resources/schools_complete.csv"

# Read our Students data into pandas
df_school = pd.read_csv(school_file)
df_school.head()

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [209]:
# The path to our CSV file
student_file = "Resources/students_complete.csv"

# Read our Students data into pandas
df_student = pd.read_csv(student_file)
df_student.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [210]:
# Merging  student with school file
all_df = pd.merge(df_school,df_student, on=('school_name'))
all_df.head()

,School ID,school_name,type,size,budget,Student ID,student_name,gender,grade,reading_score,math_score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84


### District Summary

* Create a high level snapshot (in table form) of the district's key metrics, including:
  * Total Schools
  * Total Students
  * Total Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)


In [211]:
#Total Schools
total_schools = all_df["school_name"].nunique()
#Total Students
total_students = all_df["Student ID"].count()
#Total Budget
total_budget = df_school["budget"].sum()
#Average Math Score
avg_math = all_df["math_score"].mean()
#Average Reading Score
avg_read = all_df["reading_score"].mean()
#% Passing Math
math_pass = df_student.loc[df_student["math_score"] >= 70]
count_pass_math = math_pass["math_score"].count()
per_math_pass = (count_pass_math/total_students)*100
#% Passing Reading
read_pass = df_student.loc[df_student["reading_score"] >= 70]
count_pass_read = read_pass["reading_score"].count()
per_read_pass = (count_pass_read/total_students)*100
#Overall Passing Rate (Average of the above two)
overall=(per_math_pass + per_read_pass )/2
# Make dataframe with the results
district_summary = pd.DataFrame({"Total Schools" : [total_schools],
                   "Total Students" : [total_students],
                   "Total Budget" : [total_budget],
                   "Average Math Score" :[avg_math],
                   "Average Reading Score" : [avg_read],
                    "% Passing Math" : [per_math_pass],
                   "% Passing Reading" : [per_read_pass],
                   "% Overall Passing" : [overall]
                  })
                                 
district_summary                               

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,80.393158


### School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)


In [212]:

#groups by school
school = all_df.set_index('school_name').groupby(['school_name'])
types = df_school.set_index('school_name')['type']

# sum students by school
student_school = all_df.groupby("school_name")['Student ID'].count()

# school budget
school_budget = df_school.set_index('school_name')['budget']

#student budget
student_budget = df_school.set_index('school_name')['budget']/df_school.set_index('school_name')['size']

#avg scores by school
Avg_math = school['math_score'].mean()
Avg_read = school['reading_score'].mean()

# % passing scores
pass_math = all_df[all_df['math_score'] >= 70].groupby('school_name')['Student ID'].count()/student_school 
pass_read = all_df[all_df['reading_score'] >= 70].groupby('school_name')['Student ID'].count()/student_school
overall_school = all_df[(all_df['reading_score'] >= 70) & (all_df['math_score'] >= 70)].groupby('school_name')['Student ID'].count()/student_school

school_summary = pd.DataFrame({
    "School Type": types,
    "Total Students": student_school,
    "Per Student Budget": student_budget,
    "Total School Budget": school_budget,
    "Average Math Score": Avg_math,
    "Average Reading Score": Avg_read,
    '% Passing Math': pass_math,
    '% Passing Reading': pass_read,
    "Overall Passing Rate": overall_school
})

#munging
school_summary = school_summary[['School Type', 
                          'Total Students', 
                          'Total School Budget', 
                          'Per Student Budget', 
                          'Average Math Score', 
                          'Average Reading Score',
                          '% Passing Math',
                          '% Passing Reading',
                          'Overall Passing Rate']]
#formatting
school_summary['Total Students']= school_summary ["Total Students"].map("{:,}".format)
school_summary["Total School Budget"]=school_summary["Total School Budget"].map("${:,}".format) 
school_summary["Per Student Budget"]=school_summary["Per Student Budget"].map("${:.0f}".format)
school_summary['Average Math Score']=school_summary['Average Math Score'].map("{:.1f}".format) 
school_summary['Average Reading Score']=school_summary['Average Reading Score'].map("{:.1f}".format)
school_summary["% Passing Math"]=school_summary["% Passing Math"].map("{:.1%}".format) 
school_summary["% Passing Reading"]=school_summary["% Passing Reading"].map("{:.1%}".format) 
school_summary["Overall Passing Rate"] = school_summary["Overall Passing Rate"].map("{:.1%}".format)

school_summary.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Bailey High School,District,"4,976","$3,124,928",$628,77.0,81.0,66.7%,81.9%,54.6%
Cabrera High School,Charter,"1,858","$1,081,356",$582,83.1,84.0,94.1%,97.0%,91.3%
Figueroa High School,District,"2,949","$1,884,411",$639,76.7,81.2,66.0%,80.7%,53.2%
Ford High School,District,"2,739","$1,763,916",$644,77.1,80.7,68.3%,79.3%,54.3%
Griffin High School,Charter,"1,468","$917,500",$625,83.4,83.8,93.4%,97.1%,90.6%


### Top Performing Schools (By Passing Rate)

* Create a table that highlights the top 5 performing schools based on Overall Passing Rate. Include:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)


In [213]:
Top5=school_summary.sort_values("Overall Passing Rate", ascending = False)
Top5.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Cabrera High School,Charter,"1,858","$1,081,356",$582,83.1,84.0,94.1%,97.0%,91.3%
Thomas High School,Charter,"1,635","$1,043,130",$638,83.4,83.8,93.3%,97.3%,90.9%
Griffin High School,Charter,"1,468","$917,500",$625,83.4,83.8,93.4%,97.1%,90.6%
Wilson High School,Charter,"2,283","$1,319,574",$578,83.3,84.0,93.9%,96.5%,90.6%
Pena High School,Charter,962,"$585,858",$609,83.8,84.0,94.6%,95.9%,90.5%


### Bottom Performing Schools (By Passing Rate)

* Create a table that highlights the bottom 5 performing schools based on Overall Passing Rate. Include all of the same metrics as above.

In [214]:
Bottom_Schools = school_summary.sort_values("Overall Passing Rate", ascending = True)
Bottom_Schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Rodriguez High School,District,"3,999","$2,547,363",$637,76.8,80.7,66.4%,80.2%,53.0%
Figueroa High School,District,"2,949","$1,884,411",$639,76.7,81.2,66.0%,80.7%,53.2%
Hernandez High School,District,"4,635","$3,022,020",$652,77.3,80.9,66.8%,80.9%,53.5%
Huang High School,District,"2,917","$1,910,635",$655,76.6,81.2,65.7%,81.3%,53.5%
Johnson High School,District,"4,761","$3,094,650",$650,77.1,81.0,66.1%,81.2%,53.5%


## Math Scores by Grade\*\*

* Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.


In [215]:
ninth_math = df_student.loc[df_student['grade'] == '9th'].groupby('school_name')["math_score"].mean()
tenth_math = df_student.loc[df_student['grade'] == '10th'].groupby('school_name')["math_score"].mean()
eleventh_math = df_student.loc[df_student['grade'] == '11th'].groupby('school_name')["math_score"].mean()
twelfth_math = df_student.loc[df_student['grade'] == '12th'].groupby('school_name')["math_score"].mean()

math_scores = pd.DataFrame({
        "9th": ninth_math,
        "10th": tenth_math,
        "11th": eleventh_math,
        "12th": twelfth_math
})
math_scores = math_scores[['9th', '10th', '11th', '12th']]
math_scores.index.name = "school_name"

#show and format
math_scores.style.format({'9th': '{:.1f}', 
                          "10th": '{:.1f}', 
                          "11th": "{:.1f}", 
                          "12th": "{:.1f}"})
math_scores.head()

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164


### Reading Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.


In [216]:
ninth_read = df_student.loc[df_student['grade'] == '9th'].groupby('school_name')["reading_score"].mean()
tenth_read = df_student.loc[df_student['grade'] == '10th'].groupby('school_name')["reading_score"].mean()
eleventh_read = df_student.loc[df_student['grade'] == '11th'].groupby('school_name')["reading_score"].mean()
twelfth_read = df_student.loc[df_student['grade'] == '12th'].groupby('school_name')["reading_score"].mean()
#Data framing
read_scores = pd.DataFrame({
        "9th": ninth_read,
        "10th": tenth_read,
        "11th": eleventh_read,
        "12th": twelfth_read
})
read_scores =read_scores[['9th', '10th', '11th', '12th']]
read_scores.index.name = "school_name"

#formatting
read_scores.style.format({'9th': '{:.1f}', 
                          "10th": '{:.1f}', 
                          "11th": "{:.1f}", 
                          "12th": "{:.1f}"})
read_scores.head()

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699


### Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [217]:
bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]
all_df['spending_bins'] = pd.cut(all_df['budget']/all_df['size'], bins, labels = group_names)

#group by spending
spending = all_df.groupby('spending_bins')

#calculations
avg_math = spending['math_score'].mean()
avg_read = spending['reading_score'].mean()
pass_math = all_df[all_df['math_score'] >= 70].groupby('spending_bins')['Student ID'].count()/spending['Student ID'].count() 
pass_read = all_df[all_df['reading_score'] >= 70].groupby('spending_bins')['Student ID'].count()/spending['Student ID'].count() 
overall_school = all_df[(all_df['reading_score'] >= 70) & (all_df['math_score'] >= 70)].groupby('spending_bins')['Student ID'].count()/.groupby('type')['Student ID'].count()/spending['Student ID'].count() 

#pass_read = all_df[all_df['reading_score'] >= 70].groupby('school_name')['Student ID'].count()/spending['Student ID'].count() 
#overall_school = all_df[(all_df['reading_score'] >= 70) & (all_df['math_score'] >= 70)].groupby('school_name')['Student ID'].count()/spending['Student ID'].count() 

# Data framing           
scores_by_spend = pd.DataFrame({
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,
    '% Passing Math': pass_math,
    '% Passing Reading': pass_read,
    "Overall Passing Rate": overall
            
})
            
#reorder columns
scores_by_spend = scores_by_spend[[
    "Average Math Score",
    "Average Reading Score",
    '% Passing Math',
    '% Passing Reading',
    "Overall Passing Rate"
]]

scores_by_spend.index.name = "Per Student Budget"
scores_by_spend = scores_by_spend.reindex(group_names)

#formating
#scores_by_spend['Average Math Score']=scores_by_spend['Average Math Score'].map('{:.1f}'.format) 
#scores_by_spend['Average Reading Score']=scores_by_spend['Average Reading Score'].map('{:.1f}'.format)
#scores_by_spend['% Passing Math'] = scores_by_spend['% Passing Math'].map('{:.1f}%'.format)
#scores_by_spend['% Passing Reading'] = scores_by_spend['% Passing Reading'].map('{:.1f}%'.format)
#scores_by_spend['Overall Passing Rate']=scores_by_spend['Overall Passing Rate'].map('{:.1f}%'.format)
scores_by_spend.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Per Student Budget,,,,,
<$585,83.363065,83.964039,0.937029,0.966866,80.393158
$585-615,83.529196,83.838414,0.941241,0.958869,80.393158
$615-645,78.061635,81.434088,0.714004,0.836148,80.393158
$645-675,77.049297,81.005604,0.662308,0.811094,80.393158


### Scores by School Size

* Repeat the above breakdown, but this time group schools based on a reasonable approximation of school size (Small, Medium, Large).

In [230]:
bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)" , "Large (>2000)"]
all_df['size_bins'] = pd.cut(all_df['size'], bins, labels = group_names)

#group by spending
by_size = all_df.groupby('size_bins')

#calculations 
avg_math = by_size['math_score'].mean()
avg_read = by_size['math_score'].mean()
pass_math = all_df[all_df['math_score'] >= 70].groupby('size_bins')['Student ID'].count()/by_size['Student ID'].count()
pass_read = all_df[all_df['reading_score'] >= 70].groupby('size_bins')['Student ID'].count()/by_size['Student ID'].count()
overall = all_df[(all_df['reading_score'] >= 70) & (all_df['math_score'] >= 70)].groupby('size_bins')['Student ID'].count()/by_size['Student ID'].count()
# df build            
scores_by_size = pd.DataFrame({
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,
    '% Passing Math': pass_math,
    '% Passing Reading': pass_read,
    "Overall Passing Rate": overall})
    
#reorder columns
scores_by_type = scores_by_type[[
    "Average Math Score",
    "Average Reading Score",
    '% Passing Math',
    '% Passing Reading',
    "Overall Passing Rate"
]]
scores_by_size.index.name = "Size of School"


#formating
scores_by_size.style.format({'Average Math Score': '{:.1f}', 
                              'Average Reading Score': '{:.1f}', 
                              '% Passing Math': '{:.1%}', 
                              '% Passing Reading':'{:.1%}', 
                              'Overall Passing Rate': '{:.1%}'})
scores_by_size.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Size of School,,,,,
Small (<1000),83.828654,83.828654,0.939525,0.960403,0.901368
Medium (1000-2000),83.372682,83.372682,0.936165,0.967731,0.906243
Large (>2000),77.477597,77.477597,0.686524,0.821252,0.565740


### Scores by School Type

* Repeat the above breakdown, but this time group schools based on school type (Charter vs. District).

In [219]:
by_type = all_df.groupby("type")

#calculations 
avg_math = by_type['math_score'].mean()
avg_read = by_type['math_score'].mean()
pass_math = all_df[all_df['math_score'] >= 70].groupby('type')['Student ID'].count()/by_type['Student ID'].count()
pass_read = all_df[all_df['reading_score'] >= 70].groupby('type')['Student ID'].count()/by_type['Student ID'].count()
overall = all_df[(all_df['reading_score'] >= 70) & (all_df['math_score'] >= 70)].groupby('type')['Student ID'].count()/by_type['Student ID'].count()

# df build            
scores_by_type = pd.DataFrame({
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,
    '% Passing Math': pass_math,
    '% Passing Reading': pass_read,
    "Overall Passing Rate": overall})
    
#reorder columns
scores_by_type = scores_by_type[[
    "Average Math Score",
    "Average Reading Score",
    '% Passing Math',
    '% Passing Reading',
    "Overall Passing Rate"
]]
scores_by_type.index.name = "Type of School"


#formating
scores_by_type.style.format({'Average Math Score': '{:.1f}', 
                              'Average Reading Score': '{:.1f}', 
                              '% Passing Math': '{:.1%}', 
                              '% Passing Reading':'{:.1%}', 
                              'Overall Passing Rate': '{:.1%}'})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Type of School,,,,,
Charter,83.4,83.4,93.7%,96.6%,90.6%
District,77.0,77.0,66.5%,80.9%,53.7%
